In [1]:
import numpy as np
import sympy as sp

In [7]:
from spe_decompose import decompose_spe

Lamd = 0.4
Gams = 1
Omgs = 1
Zeta = 1

temp = 0.5 # temprature
beta = 1 / temp
npsd = 2 # pade

#w_sp, eta_sp, gamma_sp, beta_sp = sp.symbols(r"\omega, \eta, \gamma, \beta", real=True)
w_sp, lamd_sp, gams_sp, zeta_sp, omgs_sp, beta_sp = sp.symbols(
r"\omega , \lambda, \gamma, \zeta, \Omega_{s}, \beta", real=True)

#phixx_sp = 2 * eta_sp * gamma_sp / (gamma_sp - sp.I * w_sp)
phiyy_sp = 2 * omgs_sp * omgs_sp * lamd_sp / (
omgs_sp * omgs_sp - w_sp * w_sp - zeta_sp * sp.I * w_sp)

spe_vib_sp = phiyy_sp

#sp_para_dict = {eta_sp: eta, gamma_sp: gam}
sp_para_dict = {lamd_sp: Lamd, gams_sp: Gams, omgs_sp: Omgs, zeta_sp: Zeta}

condition_dict = {}
para_dict = {'beta': beta}
etal, etar, etaa, expn = decompose_spe(spe_vib_sp, w_sp, sp_para_dict, para_dict,
                                       condition_dict, npsd)
print(etal)
print(etar)
#print(etaa)
print(expn)

[ 0.49722544+8.18478654e-02j  0.03534522-8.18478654e-02j
 -0.02373321-1.53626747e-18j -0.00509657+1.26894077e-19j]
[ 0.03534522+8.18478654e-02j  0.49722544-8.18478654e-02j
 -0.02373321+1.53626747e-18j -0.00509657-1.26894077e-19j]
[0.5       +0.8660254j 0.5       -0.8660254j 3.15296957+0.j
 9.74980938+0.j       ]


In [8]:
#set up spin-boson model
rho_dimension = 1 
dissipaton_modes = npsd + 2
dissipaton_cutoff = 2 # 3 excited states

rho_qubits = rho_dimension * 2 # two times of density matrix in vectorized form
dissipaton_qubits = int(np.ceil(np.log2(dissipaton_cutoff))) * dissipaton_modes # 向上取整
total_qubits = rho_qubits + dissipaton_qubits + 1

H = {'X':1, 'Z':1}
Q = {'Z':1}

alpha = expn.real
omega = expn.imag # γ=α+iΩ
zeta = np.sqrt((etal + etar) / 2)
xi = (etal - etar) / (2j * zeta)
print(alpha)
print(omega)
print(zeta)
print(xi)

[0.5        0.5        3.15296957 9.74980938]
[ 0.8660254 -0.8660254  0.         0.       ]
[0.52195094+0.07840571j 0.52195094-0.07840571j 0.        +0.15405585j
 0.        +0.07139027j]
[-0.06499751-4.32691877e-01j -0.06499751+4.32691877e-01j
 -0.        +9.97214615e-18j -0.        -1.77747018e-18j]


In [9]:
#evolution parameters
epsilon = 0.05
tau = 0.01

In [10]:
from evolution import Evolution

DQME = Evolution(rho_qubits=rho_qubits, dissipaton_qubits=dissipaton_qubits,
                 modes=dissipaton_modes, cut_off=dissipaton_cutoff,
                 H=H, Q=Q, alpha=alpha, omega=omega, zeta=zeta, xi=xi,
                 epsilon=epsilon, tau=tau)
DQME.construct_UA_circuit()
DQME.construct_US_circuit()
DQME.construct_USdagger_circuit()
display(DQME.UA_circuit.decompose().draw())
display(DQME.US_circuit.decompose().draw())
display(DQME.USdagger_circuit.decompose().draw())

░                    ░           »
q56_0: ─────────────────────────────────────░────────────────────░───────────»
       ┌───┐┌───────────┐┌───┐┌───────────┐ ░                    ░           »
q56_1: ┤ H ├┤ Rz(-0.02) ├┤ H ├┤ Rz(-0.02) ├─░────────────────────░───────────»
       ├───┤└┬──────────┤├───┤└┬──────────┤ ░                    ░           »
q56_2: ┤ H ├─┤ Rz(0.02) ├┤ H ├─┤ Rz(0.02) ├─░────────────────────░────────■──»
       └───┘ └──────────┘└───┘ └──────────┘ ░ ┌────────────────┐ ░ ┌───┐┌─┴─┐»
q56_3: ─────────────────────────────────────░─┤ Rz(-0.0086603) ├─░─┤ H ├┤ X ├»
                                            ░ ├───────────────┬┘ ░ ├───┤└───┘»
q56_4: ─────────────────────────────────────░─┤ Rz(0.0086603) ├──░─┤ H ├─────»
                                            ░ └───────────────┘  ░ └───┘     »
q56_5: ─────────────────────────────────────░────────────────────░───────────»
                                            ░                    ░           »
q56_6: ─────────────────────────────────────░────────────────────░───────────»
                                            ░                    ░           »
«                                                                           »
«q56_0: ────────────────────────────────────────────────────────────────────»
«                                                                           »
«q56_1: ────────────────────────────────────────────────────────────────────»
«                                                                           »
«q56_2: ──────────────────■───────────────────────────■─────────────────────»
«       ┌──────────────┐┌─┴─┐┌───┐┌────────────────┐┌─┴─┐┌─────────────────┐»
«q56_3: ┤ Rz(0.014766) ├┤ X ├┤ H ├┤ U(π/2,π/2,π/2) ├┤ X ├┤ Rz(-0.00064998) ├»
«       └──────────────┘└───┘└───┘└────────────────┘└───┘└─────────────────┘»
«q56_4: ────────────────────────────────────────────────────────────────────»
«                                                                           »
«q56_5: ────────────────────────────────────────────────────────────────────»
«                                                                           »
«q56_6: ────────────────────────────────────────────────────────────────────»
«                                                                           »
«                                                              »
«q56_0: ───────────────────────────────────────────────────────»
«                                                              »
«q56_1: ───────────────────────────────────────────────■───────»
«                                                      │       »
«q56_2: ──■──────────────────────■─────────────────────┼────■──»
«       ┌─┴─┐┌────────────────┐  │        ┌───┐      ┌─┴─┐  │  »
«q56_3: ┤ X ├┤ U(π/2,π/2,π/2) ├──┼────────┤ H ├──────┤ X ├──┼──»
«       └───┘└────────────────┘┌─┴─┐┌─────┴───┴─────┐└───┘┌─┴─┐»
«q56_4: ───────────────────────┤ X ├┤ Rz(0.0061121) ├─────┤ X ├»
«                              └───┘└───────────────┘     └───┘»
«q56_5: ───────────────────────────────────────────────────────»
«                                                              »
«q56_6: ───────────────────────────────────────────────────────»
«                                                              »
«                                                                             »
«q56_0: ──────────────────────────────────────────────────────────────────────»
«                                                                             »
«q56_1: ──────────────────────────■────────────────────────────────────────■──»
«                                 │                                        │  »
«q56_2: ──────────────────────────┼────────────────■───────────────────────┼──»
«       ┌────────────────┐      ┌─┴─┐       ┌───┐  │   ┌────────────────┐┌─┴─┐»
«q56_3: ┤ Rz(-0.0061121) ├──────┤ X ├───────┤ H ├──┼───┤ U(π/2,π/2,π/2) ├┤ X ├»
«       └─────┬───┬──────┘┌─────┴───┴──────┐└───┘┌─┴─┐┌┴────────────────┤└

┌───────────┐┌───────────────┐┌─────────────┐               »
q82_0: ┤ U3(π,0,π) ├┤ U(0,0,1.5208) ├┤ P(0.000125) ├───────■───────»
       └───────────┘└───────────────┘└─────────────┘       │       »
q82_1: ────────────────────────────────────────────────────┼───────»
                                                           │       »
q82_2: ────────────────────────────────────────────────────┼───────»
                                                    ┌──────┴──────┐»
q82_3: ─────────────────────────────────────────────┤ Rz(0.00025) ├»
                                                    └─────────────┘»
q82_4: ────────────────────────────────────────────────────────────»
                                                                   »
q82_5: ────────────────────────────────────────────────────────────»
                                                                   »
q82_6: ────────────────────────────────────────────────────────────»
                                                                   »
«       ┌─────────────┐               ┌───────────────┐                 »
«q82_0: ┤ P(0.000125) ├───────■───────┤ P(0.00078824) ├────────■────────»
«       └─────────────┘       │       └───────────────┘        │        »
«q82_1: ──────────────────────┼────────────────────────────────┼────────»
«                             │                                │        »
«q82_2: ──────────────────────┼────────────────────────────────┼────────»
«                             │                                │        »
«q82_3: ──────────────────────┼────────────────────────────────┼────────»
«                      ┌──────┴──────┐                         │        »
«q82_4: ───────────────┤ Rz(0.00025) ├─────────────────────────┼────────»
«                      └─────────────┘                 ┌───────┴───────┐»
«q82_5: ───────────────────────────────────────────────┤ Rz(0.0015765) ├»
«                                                      └───────────────┘»
«q82_6: ────────────────────────────────────────────────────────────────»
«                                                                       »
«       ┌──────────────┐                  ░                                  »
«q82_0: ┤ P(0.0024375) ├────────■─────────░───────────────────■──────────────»
«       └──────────────┘        │         ░                   │              »
«q82_1: ────────────────────────┼─────────░───────────────────┼──────────────»
«                               │         ░                   │              »
«q82_2: ────────────────────────┼─────────░────────■──────────┼───────────■──»
«                               │         ░ ┌───┐┌─┴─┐┌───────┴────────┐┌─┴─┐»
«q82_3: ────────────────────────┼─────────░─┤ H ├┤ X ├┤ Rz(4.5907e-05) ├┤ X ├»
«                               │         ░ ├───┤└───┘└────────────────┘└───┘»
«q82_4: ────────────────────────┼─────────░─┤ H ├────────────────────────────»
«                               │         ░ ├───┤                            »
«q82_5: ────────────────────────┼─────────░─┤ H ├────────────────────────────»
«                       ┌───────┴───────┐ ░ ├───┤                            »
«q82_6: ────────────────┤ Rz(0.0048749) ├─░─┤ H ├────────────────────────────»
«                       └───────────────┘ ░ └───┘                            »
«                                                                             »
«q82_0: ─────────────────────────────────────■────────────────────────────────»
«                                            │                                »
«q82_1: ─────────────────────────────────────┼────────────────────────────────»
«                                            │                                »
«q82_2: ─────────────────────────■───────────┼───────────■────────────────────»
«       ┌───┐┌────────────────┐┌─┴─┐┌────────┴────────┐┌─┴─┐┌────────────────┐»
«q82_3: ┤ H ├┤ U(π/2,π/2,π/2) ├┤ X ├┤ Rz(-0.00021635) ├┤ X ├┤ U(π/2,π/2,π/2) ├»
«       └───┘└────────────────┘└───┘└

┌────────────────┐┌──────────────┐                ┌──────────────┐»
q111_0: ┤ U(0,0,-1.5208) ├┤ P(-0.000125) ├───────■────────┤ P(-0.000125) ├»
        └────────────────┘└──────────────┘       │        └──────────────┘»
q111_1: ─────────────────────────────────────────┼────────────────────────»
                                                 │                        »
q111_2: ─────────────────────────────────────────┼────────────────────────»
                                          ┌──────┴───────┐                »
q111_3: ──────────────────────────────────┤ Rz(-0.00025) ├────────────────»
                                          └──────────────┘                »
q111_4: ──────────────────────────────────────────────────────────────────»
                                                                          »
q111_5: ──────────────────────────────────────────────────────────────────»
                                                                          »
q111_6: ──────────────────────────────────────────────────────────────────»
                                                                          »
«                        ┌────────────────┐                  ┌───────────────┐»
«q111_0: ───────■────────┤ P(-0.00078824) ├────────■─────────┤ P(-0.0024375) ├»
«               │        └────────────────┘        │         └───────────────┘»
«q111_1: ───────┼──────────────────────────────────┼──────────────────────────»
«               │                                  │                          »
«q111_2: ───────┼──────────────────────────────────┼──────────────────────────»
«               │                                  │                          »
«q111_3: ───────┼──────────────────────────────────┼──────────────────────────»
«        ┌──────┴───────┐                          │                          »
«q111_4: ┤ Rz(-0.00025) ├──────────────────────────┼──────────────────────────»
«        └──────────────┘                  ┌───────┴────────┐                 »
«q111_5: ──────────────────────────────────┤ Rz(-0.0015765) ├─────────────────»
«                                          └────────────────┘                 »
«q111_6: ─────────────────────────────────────────────────────────────────────»
«                                                                             »
«                           ░                                        »
«q111_0: ────────■──────────░────────────────────■───────────────────»
«                │          ░                    │                   »
«q111_1: ────────┼──────────░────────────────────┼───────────────────»
«                │          ░                    │                   »
«q111_2: ────────┼──────────░────────■───────────┼───────────■───────»
«                │          ░ ┌───┐┌─┴─┐┌────────┴────────┐┌─┴─┐┌───┐»
«q111_3: ────────┼──────────░─┤ H ├┤ X ├┤ Rz(-4.5907e-05) ├┤ X ├┤ H ├»
«                │          ░ ├───┤└───┘└─────────────────┘└───┘└───┘»
«q111_4: ────────┼──────────░─┤ H ├──────────────────────────────────»
«                │          ░ ├───┤                                  »
«q111_5: ────────┼──────────░─┤ H ├──────────────────────────────────»
«        ┌───────┴────────┐ ░ ├───┤                                  »
«q111_6: ┤ Rz(-0.0048749) ├─░─┤ H ├──────────────────────────────────»
«        └────────────────┘ ░ └───┘                                  »
«                                                                             »
«q111_0: ───────────────────────────────■─────────────────────────────────────»
«                                       │                                     »
«q111_1: ───────────────────────────────┼─────────────────────────────────────»
«                                       │                                     »
«q111_2: ────────────────────■──────────┼───────────■──────────────────────■──»
«        ┌────────────────┐┌─┴─┐┌───────┴────────┐┌─┴─┐┌────────────────┐  │  »
«q111_3: ┤ U(π/2,π/2,π/2) ├┤ X ├┤ Rz(0.0002

In [11]:
print(DQME.UA_circuit.decompose().depth())
print(DQME.US_circuit.decompose().depth())
print(DQME.USdagger_circuit.decompose().depth())

33
59
57


In [8]:
from qiskit.quantum_info import Statevector

initialstate = Statevector.from_label('0'*total_qubits)

In [9]:
steps = 1000
result = DQME.dynamics_run(initial=initialstate, steps=steps)

100%|██████████| 1000/1000 [1:40:06<00:00,  6.01s/it]   


In [10]:
from data import rdo_normal, rho_tilde

rdo_normal_list = []
for i in range(steps+1):
    rdo_normal_list.append(rdo_normal(result[i], total_qubits, rho_qubits))

rdo_normal_dict = {}
for i in range(steps+1):
    label = str(i)       
    rdo_normal_dict[label] = rdo_normal_list[i]
np.savez('rdo_normalized.npz', **rdo_normal_dict)

In [11]:
import matplotlib.pyplot as plt

population = np.zeros(shape=(steps+1), dtype=float)
population_imag = np.zeros(shape=(steps+1), dtype=float)
for i in range(steps+1):
    population[i] = (rdo_normal_list[i][0]-rdo_normal_list[i][3]).real
    population_imag[i] = (rdo_normal_list[i][0]-rdo_normal_list[i][3]).imag
    
plt.plot(np.arange(steps+1) * tau, population, label='population_real', color='blue') # plot of population
plt.plot(np.arange(steps+1) * tau, population_imag, label='population_imag', color='red')
plt.title("Population", fontsize=30, loc='center', color='purple')
plt.legend(loc='best')
plt.xlabel("Time"), plt.ylabel("Population")
plt.savefig('Population.png')
plt.close()